In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# Step 1: Install kagglehub with pandas dataset support
# !pip install kagglehub[pandas-datasets]

# Step 2: Import required modules
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Step 3: Login to Kaggle (You'll be prompted for your username and API token)
# kagglehub.login()

# Step 4: Set the filename for the dataset
file_path = "Churn_Modelling.csv"

# Step 5: Load the dataset using KaggleHub
dataset = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "shrutimechlearn/churn-modelling",
    file_path,
)

/tmp/ipython-input-2-416438808.py:15: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  dataset = kagglehub.load_dataset(


step 1. Data Preprocessing

In [3]:
X = dataset.iloc[:, 3: -1].values
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


Encoding categorical data

  Label encoding the gender column

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the 'Geography column'

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X), dtype=np.float64)

Split the dataset into tranning set and testing set

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Feature Scaling

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

step 2: Building an ANN

Initialize the ANN

In [11]:
ann = tf.keras.models.Sequential()

In [12]:
# Adding the first hidden layer
# Shallow neural network
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [13]:
# Adding the second hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [14]:
# Adding the output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

step 3: Training the ANN

In [15]:
# compiling the ANN
ann.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [16]:
from tensorflow.keras.callbacks import EarlyStopping

# Create an early stopping callback
early_stop = EarlyStopping(
    monitor='accuracy',        # Metric to monitor
    mode='max',                # We want accuracy to increase
    patience=10,               # Number of epochs to wait before stopping if no improvement
    verbose=1,
    restore_best_weights=True # Optional: reverts to best model weights
)

In [17]:
# Training
ann.fit(X_train,
    y_train,
    batch_size=32,
    epochs=100,
    callbacks=[early_stop]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.4601 - loss: 0.7425
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7923 - loss: 0.4923
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8011 - loss: 0.4505
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8102 - loss: 0.4365
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8126 - loss: 0.4258
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8226 - loss: 0.4117
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8195 - loss: 0.4189
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8273 - loss: 0.4056
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8363 - loss: 0.3869
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8400 - loss: 0.3842
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8349 - loss: 0.3902
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━

step 4: Making the predictions and evaluating the model

Use ANN model to predict if the customer
with follow information will the bank

- Geography: France
- Credit: Score, 600
- Gender: Male
- Age: 40
- Tenure: 3 years
- Balance: $ 60000

- Number of products: 2
- Does this customer have credit card ? yes
- Is this customer an Active Member: Yes
- Estimated Salary: $50000

In [18]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
[[False]]


In [19]:
# Predicting the test results
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[1516   79]
 [ 201  204]]
              precision    recall  f1-score   support

           0       0.88      0.95      0.92      1595
           1       0.72      0.50      0.59       405

    accuracy                           0.86      2000
   macro avg       0.80      0.73      0.75      2000
weighted avg       0.85      0.86      0.85      2000



In [21]:
print(accuracy_score(y_test, y_pred))

0.86
